In [19]:
!nvidia-smi


Sun Jan 18 13:41:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
%%writefile task4.cu
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>

#define N 1000000

//
// CUDA-ЯДРО СЛОЖЕНИЯ двух массивов
//
__global__ void add_arrays(float* a, float* b, float* c) {

    // вычисляем глобальный индекс элемента
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // проверяем выход за границы
    if (idx < N) {
        c[idx] = a[idx] + b[idx];
    }
}

//
// Функция ЗАПУСКА теста с заданным блоком
//
void run_test(int block_size) {

    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;

    // выделяем память на CPU
    h_a = new float[N];
    h_b = new float[N];
    h_c = new float[N];

    // инициализация массивов
    for (int i = 0; i < N; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    // выделяем память на GPU
    cudaMalloc(&d_a, N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    cudaMalloc(&d_c, N * sizeof(float));

    // копируем данные на GPU
    cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice);

    // настройка сетки и блоков
    dim3 block(block_size);
    dim3 grid((N + block_size - 1) / block_size);

    //
    // Замер времени выполнения ЯДРА
    //
    auto start = std::chrono::high_resolution_clock::now();

    add_arrays<<<grid, block>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time = end - start;

    // вывод результата
    std::cout << "Размер блока: " << block_size
              << " | Время выполнения: "
              << time.count() << " секунд\n";

    // освобождаем память
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;
}

int main() {

    std::cout << "Сравнение неоптимальной и оптимальной конфигураций\n";

    // неоптимальная конфигурация
    run_test(64);

    // оптимальная конфигурация
    run_test(256);

    return 0;
}


Writing task4.cu


In [21]:
!nvcc task4.cu -o task4



In [22]:
!./task4

Сравнение неоптимальной и оптимальной конфигураций
Размер блока: 64 | Время выполнения: 0.00788082 секунд
Размер блока: 256 | Время выполнения: 7.5529e-05 секунд
